In [5]:
#Token class is used here to create the individual tokens that were identified in lexer class
#__init__ constructor used to initialize the object with given parameters
#__repr__ special method in python that will give a string representation of the object

class Token:
    def __init__(self, token_identity, value):
        
        #the lexeme parameter (a string) is assigned to the attribute self.lexeme
        self.token_identity = token_identity
        
        #the token_code parameter (an integer) is assigned to the attribute self.token_code
        self.value = value
        
    def __repr__(self):
        return f"{self.token_identity}: {self.value}"

In [46]:
import re
#lexical analyzer used in HW3 just updated to add terminals for the grammar language for later
class Lexer: 
    tokens = [   #list of all the tokens. Each tuple defines the regular expression of each token type
        
    # Literals
    (r'\d+\.\d+', 'real_literal'),  # fractional numbers
    (r'\d+', 'natural_literal'),  # whole numbers and 0
    (r'\b(true|false)\b', 'bool_literal'),  # boolean literals
    (r"'(\\.|[^\\'])'", 'char_literal'),  # single ASCII character including escape characters
    (r'"(\\.|[^\\"])*"', 'string_literal'),  # any number of ASCII characters including escape characters
        
    # Keywords
    (r'\b(if|else|switch)\b', 'SELECTION_STATEMENT'),
    (r'\b(while|for|do)\b', 'LOOP_STATEMENT'),
    (r'\b(break|continue|case|default)\b', 'CONTROL_STATEMENT'),
    (r'\b(int|string|char|float|double|short|bool|long)\b', 'VARIABLE_DECLARATION'),

    # Special symbols
    (r'<==', 'LESS_THAN_OR_EQUAL_TO'),
    (r'>==', 'GREATER_THAN_OR_EQUAL_TO'),
    (r'==', 'EQUAL_TO'),
    (r'!==', 'NOT_EQUAL_TO'),
    (r'=', 'ASSIGNMENT'),
    (r'\+', 'ADDITION'),
    (r'\-', 'SUBTRACTION'),
    (r'\*', 'MULTIPLICATION'),
    (r'/', 'DIVISION'),
    (r'%', 'MODULO'),
    (r';', 'SEMICOLON'),
    (r'\*\*', 'EXPONENTIATION'),
    (r'\(', 'OPEN_PARENTHESIS'),   #  Symbol(s) to specify the breaking order of operations
    (r'\)', 'CLOSE_PARENTHESIS'),  #  Symbol(s) to specify the breaking order of operations
    (r'<', 'LESS_THAN'),
    (r'>', 'GREATER_THAN'),
    (r'\-!', 'UNARY_NEGATION'),
    (r'\!', 'LOGICAL_NOT'),
    (r'&&', 'LOGICAL_AND'),
    (r'\|\|', 'LOGICAL_OR'),
    (r'\{', 'OPEN_BRACE'),   # Symbol(s) of grouping code blocks
    (r'\}', 'CLOSE_BRACE'),  # Symbol(s) of grouping code blocks
    (r',', 'PARAMETER_SEPARATOR'),
    (r'\[', 'OPEN_BRACKET'),  # Symbol(s) to specify the parameters of a function
    (r'\]', 'CLOSE_BRACKET'), # Symbol(s) to specify the parameters of a function

    # Variable/function identifier
    (r'[a-zA-Z_]\w*', 'IDENTIFIER'),

    # Ignored patterns
    (r'\/\/[^\n]*', None),  # single-line comments
    (r'\/\*[\s\S]*?\*\/', None),  # block comments
    (r'\s+', None),  # whitespaces
        
    ]
        
    
    def __init__(self, input_text):
        self.input_text = input_text
    
    
    #Tokenization function will perform the lexical analysis on the given text
    #the function will return a list of the individual tokens identified from the text
    def Tokenization(self):
        the_tokens = []       
        current_position = 0     #index of current character that will be analyzed in the text
        
        while current_position < len(self.input_text):   #while loop that will iterate all the text
            matcher = None    #variable that will store the string (from text) that are a match with one of the operations in math_operantions
            for operator, token_identity in self.tokens:      #for loop that iterates all the tuples in math_operantions. Each tuples regex(operator) and token identity(token_identity)
                matcher = re.match(operator, self.input_text[current_position:])   #re.match() re module function used here to try to match the current 'operator' (regex) against the text at the current position (index). If a match is succesful then the object is assigned to matcher variable
                if matcher:   #if the above code is succesful and has a match then we continue
                    value = matcher.group(0)   #using .group(0) to group the entire substring of text that was found as a match starting from first index (0). the matched value(character(s) from text) is assigned to variable value
                    if token_identity:    #checking to make sure that matcher is not a whitespace. this would happen if 'operator' matched the whitespace regex which means the token_identity would be 'None'. If it's 'None' then we can't enter the if statement
                        the_tokens.append(Token(token_identity, value))  #creating a Token object with the matched token identity and it's value then appending to the tokens list
                    current_position = current_position + len(value)  #moving the current position to the next character(s) to be tokenized by adding it to the length of variable value so we know that we aren't tokenizing a piece of text that was already part of a substring that was already tokenized 
                    break
            
        return the_tokens    #while loop ends and the_tokens list containing all the identified tokens in the text is returned


#commands used to run this code in jupyter notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
class Parser:  #in RDA coded fashion
    
#__init__ constructor used to initialize the object 
    def __init__(self, tokenized_Tokens):
        self.tokenized_Tokens = tokenized_Tokens
        self.position = 0

        
#function that checks if the current token at the current position matches the 'token_identity'
#if there is a match then it returns True
#if it doesn't match then it returns False or returns False if there are no more tokens to process
    def matches(self, token_identity):
        if self.position < len(self.tokenized_Tokens) and self.tokenized_Tokens[self.position].token_identity == token_identity:
            self.position += 1
            return True
        return False
    
    
#beginning point of RDA process
#begins by calling the first grammar rule '<STMT>' which this rule parses the tokens based on the other grammar rules, invoking other grammar rules as needed.
#returns the final 'True' or 'False' result if the parsing is successful or not... i.e. if the tokens are in the grammar rules or not
    def Parse(self):
        print("Tokens: ")
        for token in self.tokenized_Tokens:
            print("Token's Identity: {}, Token's Value: {}".format(token.token_identity, token.value))
        print("Parsing Result = ", end = "") 
        return self.STMT()

    
# <STMT> --> <IF_STMT> | <BLOCK> | <ASSIGN> | <DECLARE> | <WHILE_LOOP> | <FUNCTIONS_DEFINITION>
    def STMT(self):
        if self.IF_STMT():
            return True
        if self.BLOCK():
            return True
        if self.ASSIGN():
            return True
        if self.DECLARE():
            return True 
        if self.WHILE_LOOP():
            return True
        if self.FUNCTIONS_DEFINITION():
            return True
        return False
    
    
# <STMT_LIST> --> { <STMT> `;` }
    def STMT_LIST(self):
        while self.matches('SEMICOLON'):
            if not self.STMT():
                return False
        return True

    
# <WHILE_LOOP> --> `while` `(` <BOOL_EXPR> `)` <BLOCK>
    def WHILE_LOOP(self):
        if not self.matches('LOOP_STATEMENT'):
            return False
        if not self.matches('OPEN_PARENTHESIS'):
            return False
        if not self.BOOL_EXPR():
            return False
        if not self.matches('CLOSE_PARENTHESIS'):
            return False
        if not self.BLOCK():
            return False
        return True
    

# <IF_STMT> --> `if` `(` <BOOL_EXPR> `)` <BLOCK> [ `else' <BLOCK> ] 
    def IF_STMT(self):
        if not self.matches('SELECTION_STATEMENT'):
            return False
        if not self.matches('OPEN_PARENTHESIS'):
            return False
        if not self.BOOL_EXPR():
            return False
        if not self.matches('CLOSE_PARENTHESIS'):
            return False
        if not self.BLOCK():
            return False
        if self.matches('SELECTION_STATEMENT'):
            if not self.BLOCK():
                return False
        return True

    
# <BLOCK> --> `{` <STMT_LIST> `}`
    def BLOCK(self):
        if not self.matches('OPEN_BRACE'):
            return False
        if not self.STMT_LIST():
            return False
        if not self.matches('CLOSE_BRACE'):
            return False
        return True


# <DECLARE> --> 'DataType' ID {',' ID}   
    def DECLARE(self):
        if not self.matches('VARIABLE_DECLARATION'):
            return False
        if not self.matches('IDENTIFIER'):
            return False
        while self.matches('COMMA'):
            if not self.matches('IDENTIFIER'):
                return False
        return True


# <ASSIGN> --> ID '=' <EXPR>
    def ASSIGN(self):
        if not self.matches('IDENTIFIER'):
            return False
        if not self.matches('ASSIGNMENT'):
            return False
        if not self.EXPR():
            return False
        return True
    

    
# <EXPR> --> [ '-' ] <TERM> {(`+`|`-`) <TERM>}
    def EXPR(self):
        if self.matches('SUBTRACTION'):
            if not self.TERM():
                return False
        else:
            if not self.TERM():
                return False
        while self.matches('ADDITION') or self.matches('SUBTRACTION'):
            if not self.TERM():
                return False
        return True

    
# <TERM> --> <FACT> {(`*`|`/`|`%`) <FACT>}
    def TERM(self):
        if not self.FACT():
            return False
        while self.matches('MULTIPLICATION') or self.matches('DIVISON') or self.matches('MODULO'):
            if not self.FACT():
                return False
        return True
    
    
# <FACT> --> ID | NATURAL_LIT | REAL_LIT | BOOL_LIT | CHAR_LIT | STRING_LIT | <FUNCTION_CALL> | `(` <EXPR> `)`
    def FACT(self):
        if (self.matches('IDENTIFIER') or self.matches('natural_literal') or self.matches('real_literal') or self.matches('bool_literal') or self.matches('char_literal') or self.matches('string_literal')):
            return True
        if self.matches('OPEN_PARENTHESIS'):
            if not self.EXPR():
                return False
            if not self.matches('CLOSE_PARENTHESIS'):
                return False
            return True
        if self.FUNCTION_CALL():
            return True
        return False
    
    
# <BOOL_EXPR> --> <BTERM> {(`>`|`<`|`>==`|`<==`) <BTERM>}
    def BOOL_EXPR(self):
        if not self.BTERM():
            return False
        while self.matches('GREATER_THAN') or self.matches('LESS_THAN') or self.matches('GREATER_THAN_OR_EQUAL_TO') or self.matches('LESS_THAN_OR_EQUAL_TO'):
            if not self.BTERM():
                return False
        return True

    
# <BTERM> --> <BAND> {(`==`|`!==`) <BAND>}
    def BTERM(self):
        if not self.BAND():
            return False
        while self.matches('EQUAL_TO') or self.matches('NOT_EQUAL_TO'):
            if not self.BAND():
                return False
        return True

    
# <BAND> --> <BOR> {`&&` <BOR>}
    def BAND(self):
        if not self.BOR():
            return False
        while self.matches('LOGICAL_AND'):
            if not self.BOR():
                return False
        return True

    
# <BOR> --> <EXPR> {`||` <EXPR>}
    def BOR(self):
        if not self.EXPR():
            return False
        while self.matches('LOGICAL_OR'):
            if not self.EXPR():
                return False
        return True
     
        
# <FUNCTION_CALL> --> ID `(` [ <EXPR> {`,` <EXPR>} ] `)`
    def FUNCTION_CALL(self):
        if not self.matches('IDENTIFIER'):
            return False
        if not self.matches('OPEN_PARENTHESIS'):
            return False
        if self.EXPR():
            while self.matches('COMMA'):
                if not self.EXPR():
                    return False
        if not self.matches('CLOSE_PARENTHESIS'):
            return False
        return True

    
# <FUNCTIONS_DEFINITION> --> `DataType` ID `(` [ `DataType` ID {`,` `DataType` ID} ] `)` <BLOCK>
    def FUNCTIONS_DEFINITION(self):
        if not self.matches('VARIABLE_DECLARATION'):
            return False
        if not self.matches('IDENTIFIER'):
            return False
        if not self.matches('OPEN_PARENTHESIS'):
            return False
        if self.matches('DataType'):
            if not self.matches('IDENTIFIER'):
                return False
            while self.matches('COMMA'):
                if not self.matches('DataType'):
                    return False
                if not self.matches('IDENTIFIER'):
                    return False
        if not self.matches('CLOSE_PARENTHESIS'):
            return False
        if not self.BLOCK():
            return False
        return True

In [48]:
class Compiler:
    
    def __init__(self):
        self.lexer = None
        self.parser = None
    
    def read_file(self, file_name):
        with open(file_name, 'r') as f:
            input = f.read()
        self.lexer = Lexer(input)
        tokens = self.lexer.Tokenization()
        self.parser = Parser(tokens)
        
    def parse(self):
        return self.parser.Parse()

compiler = Compiler()
compiler.read_file('input_str.txt')
parse_tree = compiler.parse()
print(parse_tree)

Tokens: 
Token's Identity: VARIABLE_DECLARATION, Token's Value: int
Token's Identity: IDENTIFIER, Token's Value: a
Token's Identity: PARAMETER_SEPARATOR, Token's Value: ,
Token's Identity: IDENTIFIER, Token's Value: b
Token's Identity: SEMICOLON, Token's Value: ;
Token's Identity: IDENTIFIER, Token's Value: a
Token's Identity: ASSIGNMENT, Token's Value: =
Token's Identity: natural_literal, Token's Value: 4
Token's Identity: SEMICOLON, Token's Value: ;
Token's Identity: IDENTIFIER, Token's Value: b
Token's Identity: ASSIGNMENT, Token's Value: =
Token's Identity: natural_literal, Token's Value: 3
Token's Identity: MULTIPLICATION, Token's Value: *
Token's Identity: IDENTIFIER, Token's Value: a
Token's Identity: SEMICOLON, Token's Value: ;
Token's Identity: SELECTION_STATEMENT, Token's Value: if
Token's Identity: OPEN_PARENTHESIS, Token's Value: (
Token's Identity: IDENTIFIER, Token's Value: a
Token's Identity: GREATER_THAN_OR_EQUAL_TO, Token's Value: >==
Token's Identity: IDENTIFIER, Toke